In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib.ticker import MultipleLocator
import os

In [2]:
def makeScatterPlots(x, y, UB, LB, slope, intercept, date, tStart, tStop, metric, unit, shift):
    fig, ax = plt.subplots(1,2, figsize=(12,6))

    bfX = np.arange(-50,50,1)
    bfY = intercept + (slope*bfX)

    ax[0].scatter(x, y, s=8)
    ax[0].plot(bfX, bfY, color='red', linestyle='dashed')

    ax[0].set_xlim(LB, UB)
    ax[0].set_ylim(LB, UB)

    ax[0].set_xlabel('Omni {}'.format(unit))
    ax[0].set_ylabel('Artemis {}'.format(unit))

    ax[0].xaxis.set_major_locator(MultipleLocator(5))
    ax[0].yaxis.set_major_locator(MultipleLocator(5))
    ax[0].xaxis.set_minor_locator(MultipleLocator(1))
    ax[0].yaxis.set_minor_locator(MultipleLocator(1))

    ax[0].set_title('{} between {}-{} UT'.format(date, tStart, tStop), fontsize=10)
    ax[0].text(.01, .99, 'Slope = {}'.format(round(slope, 2)), ha='left', va='top', transform=ax[0].transAxes)


    ax[1].plot([j for j in range(0,60)], x, label='OMNI')
    ax[1].plot([j for j in range(0,60)], y, label='ARTEMIS')
    ax[1].legend()

    ax[1].set_xlim(0, 59)
    ax[1].set_ylim(LB, UB)

    ax[1].set_xlabel('Minute of hour')
    ax[1].set_ylabel('{}'.format(unit))

    ax[1].xaxis.set_minor_locator(MultipleLocator(1))
    ax[1].xaxis.set_major_locator(MultipleLocator(5))

    ax[1].text(.01, .99, 'Shift = {} Min.'.format(round(shift, 2)), ha='left', va='top', transform=ax[1].transAxes)


    plt.suptitle('ARTEMIS {} as a function of observed OMNI'.format(metric, metric), y=0.95)


In [3]:
# Set up ARTEMIS and OMNI raw file directories and list of files
projDir ='/Volumes/Research'
artemisDataDirectory = os.path.join(projDir, 'Solar-Wind-Reliability/output-data/GSE/Artemis/')
omniDataDirectory = os.path.join(projDir, 'Solar-Wind-Reliability/output-data/GSE/Omni/')
omniFileList = sorted(filter(lambda x: os.path.isfile(os.path.join(omniDataDirectory, x)), os.listdir(omniDataDirectory)))
artemisFileList = sorted(filter(lambda x: os.path.isfile(os.path.join(artemisDataDirectory, x)), os.listdir(artemisDataDirectory)))

omniFileList.remove('.DS_Store')
artemisFileList.remove('.DS_Store')

metricDirectory = os.path.join(projDir, 'Solar-Wind-Reliability/output-data/hourly-correlations/')
metricFolders = sorted(filter(lambda x: os.path.isdir(os.path.join(metricDirectory, x)), os.listdir(metricDirectory)))

In [4]:
print(omniFileList)

['Omni_2011-10-24_18-10.csv', 'Omni_2011-12-28_16-20.csv', 'Omni_2013-01-17_10-30.csv', 'Omni_2014-02-27_16-30.csv', 'Omni_2014-02-27_18-30.csv', 'Omni_2014-04-05_19-30.csv', 'Omni_2015-03-16_03-50.csv', 'Omni_2015-03-17_04-20.csv', 'Omni_2015-04-20_20-10.csv', 'Omni_2015-06-12_17-10.csv', 'Omni_2016-06-05_05-00.csv', 'Omni_2016-08-02_12-15.csv', 'Omni_2017-09-15_12-00.csv', 'Omni_2017-09-27_04-30.csv', 'Omni_2018-09-13_00-00.csv', 'Omni_2018-10-09_04-00.csv', 'Omni_2019-11-27_13-00.csv']


In [7]:
#keys = ['BX_GSE', 'BY_GSE', 'BZ_GSE', 'Vx', 'Vy', 'Vz', 'proton_density', 'T']
#units = [r'$B_x$ (nT)', r'$B_y$ (nT)', r'$B_z$ (nT)', r'$V_x$ (km/s)', r'$V_y$ (km/s)', r'$V_z$ (km/s)', r'Density ($cm^{-3}$)', 'Temperature (K)']

keys = ['BY_GSE', 'BZ_GSE']
units = [r'$B_y$ (nT)', r'$B_z$ (nT)']

#for l in range(len(omniFileList)):
for l in range(16, 17, 1):
    artemisData = pd.read_csv(os.path.join(artemisDataDirectory, artemisFileList[l]), delimiter=',', header=0, index_col=0)
    omniData = pd.read_csv(os.path.join(omniDataDirectory, omniFileList[l]), delimiter=',', header=0, index_col=0)
    artemisData['Time'] = pd.to_datetime(artemisData['Time'], format='%Y-%m-%d %H:%M:%S')
    omniData['Time'] = pd.to_datetime(omniData['Time'], format='%Y-%m-%d %H:%M:%S')

    numWindows = len(omniData)-60 # Number of 1-hour blocks in the data set
    for n in range(numWindows):
        aStart = (artemisData.loc[artemisData['Time'] == omniData['Time'][n]]).index[0]
        aStop = (artemisData.loc[artemisData['Time'] == omniData['Time'][n+60]]).index[0]

        for k, u in zip(keys, units):
            shiftVals = pd.read_csv(os.path.join(metricDirectory, metricFolders[l], k, 'timeshifts.csv'), delimiter=',', header=0, index_col=0)
            slopeVals = pd.read_csv(os.path.join(metricDirectory, metricFolders[l], k, 'metrics.csv'), delimiter=',', header=0, index_col=0)

            shift = shiftVals['Pearson'][n]
            slopes = slopeVals['Slope'][n]
            intercepts = slopeVals['Intercept'][n]

            upperBound = max([max(omniData[k]), max(artemisData[k])]) + 5
            lowerBound = min([min(omniData[k]), min(artemisData[k])]) - 5

            makeScatterPlots(omniData[k][n:(n+60)], artemisData[k][aStart-shift:aStop-shift], upperBound, lowerBound, slopes, intercepts, omniData['Time'][n].strftime('%Y-%m-%d'), omniData['Time'][n].strftime('%H:%M'), omniData['Time'][n+59].strftime('%H:%M'), k, u, shift)

            if os.path.exists(os.path.join('/Volumes/Research/Solar-Wind-Reliability/output-data/scatterplots/{}/{}/'.format(omniData['Time'][n].strftime('%Y-%m-%d'), k))):
                plt.savefig(os.path.join('/Volumes/Research/Solar-Wind-Reliability/output-data/scatterplots/{}/{}/{}.png'.format(omniData['Time'][n].strftime('%Y-%m-%d'), k, omniData['Time'][n].strftime('%H-%M'))), dpi=300)
            else:
                os.makedirs(os.path.join('/Volumes/Research/Solar-Wind-Reliability/output-data/scatterplots/{}/{}/'.format(omniData['Time'][n].strftime('%Y-%m-%d'), k)))
                plt.savefig(os.path.join('/Volumes/Research/Solar-Wind-Reliability/output-data/scatterplots/{}/{}/{}.png'.format(omniData['Time'][n].strftime('%Y-%m-%d'), k, omniData['Time'][n].strftime('%H-%M'))), dpi=300)
            plt.close()